## Clean Rental Rate Notebook

This notebook aims to clean the rental rates for the given data.

First we perform necessary imports.

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import json
import re

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Crime Curated")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .config("spark.driver.port", "6066")
    .getOrCreate()
)

FILE_PATH = '../data/raw/example.json'

your 131072x1 screen size is bogus. expect trouble
24/09/08 13:51:33 WARN Utils: Your hostname, TABLET-K04EDL4Q resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/08 13:51:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 13:51:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/08 13:51:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


All the functions used are defined below:

In [112]:
def extract_number(text, keyword):
    """
    Takes a string input and extracts the numeric value, casting it to an int.
    """
    match = re.search(rf'(\d+)\s*{keyword}', text, re.IGNORECASE)
    return int(match.group(1)) if match else None

def find_largest_number(text, number_pattern):
    """
    Finds the largest number based on a number pattern in a given string.
    """
    number_pattern = r'\$?\d+[\d,]*(?:\.\d+)?'
    matches = re.findall(number_pattern, text)

    if matches:
        numbers = [float(match.replace('$', '').replace(',', '')) for match in matches]
        return max(numbers)
    return None

Now load the data into a data frame.

In [5]:
# 0: define the schema for the dataframe
schema = StructType([
    StructField("url", StringType(), True),
    StructField("cost_text", StringType(), True),
    StructField("bedrooms", IntegerType(), True),
    StructField("bathrooms", IntegerType(), True)
])

# 1: read the file
with open(FILE_PATH, 'r') as fp:
    data = json.load(fp)

# 2: extract the cost, along with the number of bedrooms and bathrooms
extracted_data = []
for url, entity in data.items():
    cost_text = entity.get("cost_text", "")
    rooms = entity.get("rooms", [])
    
    bedrooms = None
    bathrooms = None
    for room in rooms:
        if 'bed' in room.lower():
            bedrooms = extract_number(room, 'bed')
        elif 'bath' in room.lower():
            bathrooms = extract_number(room, 'bath')
    
    extracted_data.append((url, cost_text, bedrooms, bathrooms))

# 3: create the dataframe from this data
df = spark.createDataFrame(extracted_data, schema=schema)

# verify the dataframe is working as expected
print(f"Dataframe of {df.count()} rows.")
df.limit(5)

Dataframe of 977 rows.


url,cost_text,bedrooms,bathrooms
https://www.domai...,Price on Application,9,9
https://www.domai...,"$38,000 p.a. Incl...",NULL,NULL
https://www.domai...,"$12,500 pw",4,4
https://www.domai...,"$9,999 pw",5,5
https://www.domai...,"$5,750.00",2,2


First, we will do some exploratory analysis. 

1. How many don't have the price included at all?

In [117]:
# df.where(df['cost_text'])

COST_COLUMNS = ['unknown_rate', 'per_month', 'per_week', 'per_year']

number_pattern = r'\$?\d+[\d,]*(\.\d+)?'

per_annum_text = r'p\.?a\.?|per annum|py|p\.y\.|per year|yr|year|yearly|p/y'
per_calendar_month_text = r'p\.?m\.?|per month|p\.?c\.?m\.?|month|monthly|mth|p/m'
per_week_text = r'p\.?w\.?|per week|week|weekly|p/w|wk|w'

between_amount_and_rate = f'\s*/?\s*'

# match if just a number
df_extracted = df.withColumn("unknown_rate", F.regexp_extract(F.col("cost_text"), fr'^{number_pattern}$', 0)) \

# match with per week strings
df_extracted = df_extracted.withColumn("per_week", F.regexp_extract(F.col("cost_text"), fr'(?i){number_pattern}{between_amount_and_rate}({per_week_text})',
                                                          0))

# match with per year strings
df_extracted = df_extracted.withColumn("per_year", F.regexp_extract(F.col("cost_text"), fr'(?i){number_pattern}{between_amount_and_rate}({per_annum_text})',
                                                          0))

# match with per month strings
df_extracted = df_extracted.withColumn("per_month", F.regexp_extract(F.col("cost_text"), fr'(?i){number_pattern}{between_amount_and_rate}({per_calendar_month_text})',
                                                          0))

# finally match with the largest number in the text
find_largest_number_udf = F.udf(lambda text: find_largest_number(text, number_pattern), FloatType())

# convert columns to floats
for col_name in COST_COLUMNS:
    df_extracted = df_extracted.withColumn(col_name, F.regexp_replace(F.col(col_name), r'[^\d.]', '')) \
    .withColumn(col_name, F.col(col_name).cast("float"))

# find and save the largest number to a new column
df_extracted = df_extracted.withColumn("unknown_rate_within_text", find_largest_number_udf(F.col("cost_text"))) \
                            .withColumn(
                                "unknown_rate", 
                                F.coalesce(F.col("unknown_rate"), F.col("unknown_rate_within_text"))
                            ).drop("unknown_rate_within_text")



# df_extracted.where(F.col('unknown_rate_within_text').isNotNull())

df_extracted.limit(20)

url,cost_text,bedrooms,bathrooms,unknown_rate,per_week,per_year,per_month
https://www.domai...,Price on Application,9,9,NULL,NULL,NULL,NULL
https://www.domai...,"$38,000 p.a. Incl...",NULL,NULL,38000.0,NULL,NULL,NULL
https://www.domai...,"$12,500 pw",4,4,12500.0,12500.0,NULL,NULL
https://www.domai...,"$9,999 pw",5,5,9999.0,9999.0,NULL,NULL
https://www.domai...,"$5,750.00",2,2,5750.0,NULL,NULL,NULL
https://www.domai...,Fully Furn - $500...,4,4,5000.0,5000.0,NULL,NULL
https://www.domai...,5000,2,2,5000.0,NULL,NULL,NULL
https://www.domai...,"$4,000 per week",3,2,4000.0,4000.0,NULL,NULL
https://www.domai...,$3900 pw Furnished,4,3,3900.0,3900.0,NULL,NULL
https://www.domai...,"$3,800.00 per week",5,6,3800.0,3800.0,NULL,NULL


Checking to see how many entries have unknown rates.

In [121]:
KNOWN_RATE_COLUMNS = ['per_month', 'per_week', 'per_year']

unknown_rate_condition = F.col(KNOWN_RATE_COLUMNS[0]).isNull()

for col in KNOWN_RATE_COLUMNS[1:]:
    unknown_rate_condition = unknown_rate_condition & F.col(col).isNull()

# apply the filter (+ also filter out where no numbers in text for clarity)
print(f"There are: {df_extracted.filter(unknown_rate_condition).count()} of {df_extracted.count()} total columns with unknown rates.")

There are: 418 of 977 total columns with unknown rates.


Since this is a large portion of rents, a future step is to create a model to predict if the rate is monthly, weekly or per year. This will be trained as a supervised model on the known rate types.

In [ ]:
# for now, merge everything into one column 

def yearly_to_weekly_rate(yearly_rate):
    """ 
    Converts a yearly rental rate to a weekly rate.
    """
    return yearly_rate / 52.14

def monthly_to_weekly_rate(monthly_rate):
    """
    Converts a monthly rental rate to a weekly rate.
    """
    return yearly_to_weekly_rate(monthly_rate * 12)

df_extracted = df_extracted.withColumn("unknown_rate_within_text", find_largest_number_udf(F.col("cost_text"))) \
                            .withColumn(
                                "unknown_rate", 
                                F.coalesce(F.col("unknown_rate"), F.col("unknown_rate_within_text"))
                            ).drop("unknown_rate_within_text")

This section is for debugging purposes. Use this to see if there's any phrases still not being matched.

In [110]:
# check for rows where all is null
null_condition = F.col(COST_COLUMNS[0]).isNull()

for col in COST_COLUMNS[1:]:
    null_condition = null_condition & F.col(col).isNull()

# apply the filter (+ also filter out where no numbers in text for clarity)
df_view = df_extracted.filter(null_condition).filter(F.col("cost_text").rlike(r'\d'))

print("Length: ", df_view.count())
df_view.show(20, truncate=False)

Length:  0
+---+---------+--------+---------+------------+--------+--------+---------+
|url|cost_text|bedrooms|bathrooms|unknown_rate|per_week|per_year|per_month|
+---+---------+--------+---------+------------+--------+--------+---------+
+---+---------+--------+---------+------------+--------+--------+---------+



In [24]:
df.take(2)[1]

Row(url='https://www.domain.com.au/667-glenhuntly-road-caulfield-vic-3162-11598047', cost_text='$38,000 p.a. Incl. Outgoings + GST', bedrooms=None, bathrooms=None)